<a href="https://colab.research.google.com/github/kevin-oliveira178/Felicidade_Mudial/blob/main/Felicidade_mundial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# importação de bibliotecas

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



# tratamento dos dados

In [8]:
df = pd.read_csv('/2019.csv')
colunas = list(df.columns)
colunas

['Overall rank',
 'Country or region',
 'Score',
 'GDP per capita',
 'Social support',
 'Healthy life expectancy',
 'Freedom to make life choices',
 'Generosity',
 'Perceptions of corruption']

In [ ]:
#criação das colunas adicionais

# parte 1


# parte 2

# parte 3